In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import copy
from model import EfficientNet_B0

%load_ext autoreload
%autoreload 2

In [2]:
batch = 256

transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classifier = EfficientNet_B0().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

In [ ]:
best_epoch = 0
best_loss = 999
best_weights = copy.deepcopy(classifier.state_dict())

save_dir = 'saved_models/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
        
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() * len(data)
    
    running_loss /= len(trainset)    
    if best_loss < running_loss:
        best_loss = running_loss
        best_weights = copy.deepcopy(classifier.state_dict())
        best_epoch = epoch
        
    print('[epoch %d] loss: %.3f' % (epoch + 1, running_loss))
    torch.save(classifier.state_dict(), os.path.join(save_dir, 'epoch_{}.pth'.format(epoch)))
    
print('Finished Training')
print('best epoch: {}'.format(best_epoch))
torch.save(best_weights, os.path.join(save_dir, 'best.pth'))

[epoch 1] loss: 0.018
[epoch 2] loss: 0.014
[epoch 3] loss: 0.012
[epoch 4] loss: 0.011
[epoch 5] loss: 0.011
[epoch 6] loss: 0.010
[epoch 7] loss: 0.010
[epoch 8] loss: 0.009
[epoch 9] loss: 0.009
[epoch 10] loss: 0.008
[epoch 11] loss: 0.008
[epoch 12] loss: 0.007
[epoch 13] loss: 0.007
[epoch 14] loss: 0.007
[epoch 15] loss: 0.006
[epoch 16] loss: 0.006
[epoch 17] loss: 0.006
[epoch 18] loss: 0.005
[epoch 19] loss: 0.005
[epoch 20] loss: 0.005
[epoch 21] loss: 0.004
[epoch 22] loss: 0.004
[epoch 23] loss: 0.004
[epoch 24] loss: 0.004
[epoch 25] loss: 0.003
[epoch 26] loss: 0.003
[epoch 27] loss: 0.003
[epoch 28] loss: 0.003
[epoch 29] loss: 0.003
[epoch 30] loss: 0.002
[epoch 31] loss: 0.002
[epoch 32] loss: 0.002
[epoch 33] loss: 0.002
[epoch 34] loss: 0.002
[epoch 35] loss: 0.002
[epoch 36] loss: 0.002
[epoch 37] loss: 0.002
[epoch 38] loss: 0.002
[epoch 39] loss: 0.001
[epoch 40] loss: 0.001
[epoch 41] loss: 0.001
[epoch 42] loss: 0.001
[epoch 43] loss: 0.001
[epoch 44] loss: 0.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[0:4]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
outputs = classifier(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

## Luke's pretrianed model